In [1]:
import pandas as pd
import re
import os

# Set the path as tangible
os.chdir('..')

# Check current path 
print(os.getcwd())


c:\Users\musta\Documents\Python Projects\tangible


In [10]:
# Read the text file
file_path = 'data/contractions/102-Ahmet Ümit _ İstanbul Hatırası.txt'
base_name = os.path.basename(file_path)
document_name = os.path.splitext(base_name)[0]
print("Document: " + document_name)


with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()


Document: 102-Ahmet Ümit _ İstanbul Hatırası


In [11]:
# 1) Küçük harfe dönüştür
text = text.lower()

# 2) Noktalama işaretlerini sil (regex ile)
#    Burada ^\w\s -> Alfanumerik karakterler ve boşluklar dışındaki her şeyi temizleyecek.
text = re.sub(r'[^\w\s]', '', text)

# 3) Sayı vb. temizlemek isterseniz ekstra regexler ekleyebilirsiniz
# (Örn. text = re.sub(r'\d+', '', text) gibi)
text = re.sub(r'\d+', '', text)

# Tokenization
tokens = text.split()  # Boşluk karakterine göre ayırır


In [12]:
# Create data frame
df = pd.DataFrame(tokens, columns=['word'])
df['count'] = 1

# Gruplayarak frekans (count) değerini hesapla
word_counts = df.groupby('word')['count'].sum().reset_index()
# Dosya ismini ekle
word_counts['source'] = document_name

# Frekansına göre büyükten küçüğe sırala
word_counts = word_counts.sort_values('count', ascending=False)

# İlk 20 kelimeyi görüntüleyelim
print(word_counts.head(20))


        word  count                              source
3709     bir   3580  102-Ahmet Ümit _ İstanbul Hatırası
4077      bu   1690  102-Ahmet Ümit _ İstanbul Hatırası
5357      de   1534  102-Ahmet Ümit _ İstanbul Hatırası
754      ama   1190  102-Ahmet Ümit _ İstanbul Hatırası
5106      da   1113  102-Ahmet Ümit _ İstanbul Hatırası
17469     ne   1010  102-Ahmet Ümit _ İstanbul Hatırası
9102    gibi    950  102-Ahmet Ümit _ İstanbul Hatırası
17710      o    870  102-Ahmet Ümit _ İstanbul Hatırası
6057    diye    675  102-Ahmet Ümit _ İstanbul Hatırası
12372   için    619  102-Ahmet Ümit _ İstanbul Hatırası
5108    daha    567  102-Ahmet Ümit _ İstanbul Hatırası
12699  kadar    482  102-Ahmet Ümit _ İstanbul Hatırası
16896     mi    457  102-Ahmet Ümit _ İstanbul Hatırası
3343     ben    439  102-Ahmet Ümit _ İstanbul Hatırası
14312     ki    431  102-Ahmet Ümit _ İstanbul Hatırası
5737   değil    427  102-Ahmet Ümit _ İstanbul Hatırası
533      ali    421  102-Ahmet Ümit _ İstanbul H

In [13]:
# CSV dosyasına kaydet
csv_path = 'data/contractions/word_frequencies.csv'
if not os.path.exists(csv_path):
    # CSV yoksa oluştur (header dahil)
    word_counts.to_csv(csv_path, index=False, encoding='utf-8')
else:
    # CSV varsa oku ve birleştir (append)
    existing_df = pd.read_csv(csv_path)
    combined_df = pd.concat([existing_df, word_counts], ignore_index=True)
    combined_df.to_csv(csv_path, index=False, encoding='utf-8')
